# 📤 Módulo 01c - Carga a SQL Server

**Nombre del archivo:** `01c_load_to_sql.ipynb`  
**Objetivo:** Cargar **30 tablas** (INE + EUROSTAT) desde pickle cache a SQL Server  
**Base de datos destino:** SQL Server (configuración en `.env`)  
**Fecha de última edición:** 2025-11-16  
**Autor:** Mario (databamario)  

**Prerequisitos:**  
- `01a_extract_transform_INE.ipynb` ejecutado (14 archivos pickle)
- `01b_extract_transform_EUROSTAT.ipynb` ejecutado (14 archivos pickle)
- Base de datos SQL Server creada y configurada en `.env`

**Nota:** Consulta [`docs/DICCIONARIO_DATOS.md`](../../docs/DICCIONARIO_DATOS.md) para documentación completa de las tablas cargadas.

---

In [2]:
import pickle
import pyodbc
from sqlalchemy import create_engine
import pandas as pd
import urllib.parse
from pathlib import Path
# Compute project_root robustly; in notebooks the current working directory may vary
def _find_project_root():
    p = Path.cwd()
    while p != p.parent:
        if (p / '.git').exists() or (p / 'README.md').exists():
            return p
        p = p.parent
    return Path.cwd()
project_root = _find_project_root()
CACHE_DIR = project_root / 'outputs' / 'pickle_cache'

print('📂 Cargando DataFrames desde pickle...')


📂 Cargando DataFrames desde pickle...


### 📥 Cargar DataFrames INE y Eurostat

In [3]:
# Cargar todos los pickles
def cargar_pickle(nombre):
    ruta = CACHE_DIR / f'{nombre}.pkl'
    with open(ruta, 'rb') as f:
        return pickle.load(f)

# DataFrames INE
df_ipc_anual = cargar_pickle('df_ipc_anual')
df_umbral_limpio = cargar_pickle('df_umbral_limpio')
df_carencia_limpio = cargar_pickle('df_carencia_material')
df_arope_edad_sexo_limpio = cargar_pickle('df_arope_edad_sexo')
df_arope_hogar_limpio = cargar_pickle('df_arope_hogar')
df_arope_laboral_limpio = cargar_pickle('df_arope_laboral')
df_gini_ccaa_limpio = cargar_pickle('df_gini_ccaa')
df_renta_decil_limpio = cargar_pickle('df_renta_decil')
df_pob_edad_sexo_nacion_limpio = cargar_pickle('df_poblacion')
df_pob_edad_sexo_ccaa_limpio = cargar_pickle('df_poblacion_ccaa_edad')
df_arope_ccaa_limpio = cargar_pickle('df_arope_ccaa')
df_gasto_quintil_limpio = cargar_pickle('df_epf_gasto')
df_ipc_sectorial_limpio = cargar_pickle('df_ipc_sectorial')

# DataFrames Eurostat
df_gini_es = cargar_pickle('df_gini_es')
df_gini_ue27 = cargar_pickle('df_gini_ue27')
df_gini_todos = cargar_pickle('df_gini_todos')
df_arop_es = cargar_pickle('df_arop_es')
df_arop_ue27 = cargar_pickle('df_arop_ue27')
df_arop_eu_todos = cargar_pickle('df_arop_eu_todos')
df_s80s20_es = cargar_pickle('df_s80s20_es')
df_s80s20_ue27 = cargar_pickle('df_s80s20_ue27')
df_s80s20_todos = cargar_pickle('df_s80s20_todos')
df_gap_es = cargar_pickle('df_gap_es')
df_gap_ue27 = cargar_pickle('df_gap_ue27')
df_gap_todos = cargar_pickle('df_gap_todos')
df_impacto_redistrib_es = cargar_pickle('df_impacto_redistrib_es')
df_impacto_redistrib_ue27 = cargar_pickle('df_impacto_redistrib_ue27')

print('✅ Todos los DataFrames cargados correctamente')


✅ Todos los DataFrames cargados correctamente


In [4]:
# Pre-check: ensure critical pickles are non-empty before attempting to write to SQL
ABORT_ON_EMPTY_PICKLE = True  # Set to False to only warn and continue
CRITICAL_PICKLES = [
    'df_ipc_sectorial.pkl',  # IPC sectorial is critical for inflation analysis
    'df_gini_ccaa.pkl',       # Gini CCAA used heavily in validations
    'df_epf_gasto.pkl'        # EPF gasto for inflation differential computations
]
CRITICAL_PICKLES_COLS = {
    'df_ipc_sectorial.pkl': ['Anio', 'Categoria_ECOICOP', 'Inflacion_Sectorial_%'],
    'df_gini_ccaa.pkl': ['Territorio', 'Anio', 'Gini', 'S80/S20'],
    'df_epf_gasto.pkl': ['Anio', 'Quintil', 'Grupo_Gasto', 'Valor']
}

from pathlib import Path
missing_or_empty = []
for p in CRITICAL_PICKLES:
    ppath = CACHE_DIR / p
    if not ppath.exists():
        print(f"⚠️ Crítico: archivo pickle faltante: {ppath}")
        missing_or_empty.append(str(ppath))
    else:
        try:
            import pandas as pd
            df = pd.read_pickle(ppath)
            if df is None or df.empty:
                print(f"⚠️ Crítico: pickle vacío: {ppath} (shape: {None if df is None else df.shape})")
                missing_or_empty.append(str(ppath))
            else:
                print(f"✅ Crítico OK: {ppath} ({df.shape[0]} rows x {df.shape[1]} cols)")
                expected_cols = CRITICAL_PICKLES_COLS.get(p, [])
                if expected_cols:
                    missing_cols = [c for c in expected_cols if c not in df.columns]
                    if missing_cols:
                        print(f"⚠️ Pickle {ppath} lacks expected columns: {missing_cols}")
                        missing_or_empty.append(str(ppath))
        except Exception as e_read:
            print(f"⚠️ No se pudo leer pickle {ppath}: {e_read}")
            missing_or_empty.append(str(ppath))

if missing_or_empty:
    msg = f'Archivos críticos faltantes o vacíos: {missing_or_empty}'
    print('\nERROR PRE-CHECK: ' + msg)
    if ABORT_ON_EMPTY_PICKLE:
        raise SystemExit(msg)
    else:
        print('Continuando a pesar de archivos críticos faltantes (ABORT_ON_EMPTY_PICKLE=False)')


✅ Crítico OK: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\outputs\pickle_cache\df_ipc_sectorial.pkl (1248 rows x 5 cols)
✅ Crítico OK: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\outputs\pickle_cache\df_gini_ccaa.pkl (340 rows x 4 cols)
✅ Crítico OK: C:\Users\mario\Desktop\Projects\desigualdad_social_etl\outputs\pickle_cache\df_epf_gasto.pkl (5616 rows x 5 cols)


### 📋 Diccionario de Tablas a Cargar (27 tablas total)

In [5]:
dataframes_a_cargar = {
    # Tablas del INE (13 tablas)
    'INE_IPC_Nacional': df_ipc_anual,
    'INE_Umbral_Pobreza_Hogar': df_umbral_limpio,
    'INE_Carencia_Material_Decil': df_carencia_limpio,
    'INE_AROPE_Edad_Sexo': df_arope_edad_sexo_limpio,
    'INE_AROPE_Hogar': df_arope_hogar_limpio,
    'INE_AROPE_Laboral': df_arope_laboral_limpio,
    'INE_AROPE_CCAA': df_arope_ccaa_limpio,
    'INE_Gini_S80S20_CCAA': df_gini_ccaa_limpio,
    'INE_Renta_Media_Decil': df_renta_decil_limpio,
    'INE_Poblacion_Edad_Sexo_Nacionalidad': df_pob_edad_sexo_nacion_limpio,
    'INE_Poblacion_Edad_Sexo_CCAA': df_pob_edad_sexo_ccaa_limpio,
    'INE_Gasto_Medio_Hogar_Quintil': df_gasto_quintil_limpio,
    'INE_IPC_Sectorial_ECOICOP': df_ipc_sectorial_limpio,
    
    # Tablas de Eurostat (14 tablas: España, UE27, Ranking para 4 indicadores + 2 impacto redistributivo)
    'EUROSTAT_Gini_Espana': df_gini_es,
    'EUROSTAT_Gini_UE27': df_gini_ue27,
    'EUROSTAT_Gini_Ranking': df_gini_todos,
    
    'EUROSTAT_AROP_Espana': df_arop_es,
    'EUROSTAT_AROP_UE27': df_arop_ue27,
    'EUROSTAT_AROP_Ranking': df_arop_eu_todos,
                                                                                                                                                                                                                                                                    
    'EUROSTAT_S80S20_Espana': df_s80s20_es,
    'EUROSTAT_S80S20_UE27': df_s80s20_ue27,
    'EUROSTAT_S80S20_Ranking': df_s80s20_todos,
    
    'EUROSTAT_Brecha_Pobreza_Espana': df_gap_es,
    'EUROSTAT_Brecha_Pobreza_UE27': df_gap_ue27,
    'EUROSTAT_Brecha_Pobreza_Ranking': df_gap_todos,
    
    'EUROSTAT_Impacto_Redistributivo_Espana': df_impacto_redistrib_es,
    'EUROSTAT_Impacto_Redistributivo_UE27': df_impacto_redistrib_ue27
}

print(f"Preparadas {len(dataframes_a_cargar)} tablas para cargar en SQL Server.")
print("\n📊 Tablas del INE (13 tablas):")
for i, tabla in enumerate([k for k in dataframes_a_cargar.keys() if k.startswith('INE')], 1):
    print(f"   {i}. {tabla}")

print("\n🇪🇺 Tablas de Eurostat (14 tablas - España + UE27 + Ranking):")
for i, tabla in enumerate([k for k in dataframes_a_cargar.keys() if k.startswith('EUROSTAT')], 1):
    print(f"   {i}. {tabla}")

Preparadas 27 tablas para cargar en SQL Server.

📊 Tablas del INE (13 tablas):
   1. INE_IPC_Nacional
   2. INE_Umbral_Pobreza_Hogar
   3. INE_Carencia_Material_Decil
   4. INE_AROPE_Edad_Sexo
   5. INE_AROPE_Hogar
   6. INE_AROPE_Laboral
   7. INE_AROPE_CCAA
   8. INE_Gini_S80S20_CCAA
   9. INE_Renta_Media_Decil
   10. INE_Poblacion_Edad_Sexo_Nacionalidad
   11. INE_Poblacion_Edad_Sexo_CCAA
   12. INE_Gasto_Medio_Hogar_Quintil
   13. INE_IPC_Sectorial_ECOICOP

🇪🇺 Tablas de Eurostat (14 tablas - España + UE27 + Ranking):
   1. EUROSTAT_Gini_Espana
   2. EUROSTAT_Gini_UE27
   3. EUROSTAT_Gini_Ranking
   4. EUROSTAT_AROP_Espana
   5. EUROSTAT_AROP_UE27
   6. EUROSTAT_AROP_Ranking
   7. EUROSTAT_S80S20_Espana
   8. EUROSTAT_S80S20_UE27
   9. EUROSTAT_S80S20_Ranking
   10. EUROSTAT_Brecha_Pobreza_Espana
   11. EUROSTAT_Brecha_Pobreza_UE27
   12. EUROSTAT_Brecha_Pobreza_Ranking
   13. EUROSTAT_Impacto_Redistributivo_Espana
   14. EUROSTAT_Impacto_Redistributivo_UE27


### 🔌 Configuración de Conexión SQL Server

In [6]:
# Importar configuración desde utils (usa .env)
import sys
from pathlib import Path
import pandas as pd
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

from utils.config import DB_CONNECTION_STRING

try:
    # Crear el "engine" de SQLAlchemy usando la configuración del .env
    quoted_conn_str = urllib.parse.quote_plus(DB_CONNECTION_STRING)
    engine = create_engine(f'mssql+pyodbc:///?odbc_connect={quoted_conn_str}')
    
    # Probar la conexión
    connection = engine.connect()
    print(f"✅ Conexión exitosa a SQL Server")
    print(f"   Cadena de conexión cargada desde .env")
    connection.close()
    
except Exception as e:
    print(f"❌ Error de conexión: {e}")
    raise


✅ Conexión exitosa a SQL Server
   Cadena de conexión cargada desde .env


In [7]:
# 🔧 Función de Normalización Master para SQL
def normalize_for_sql(df: pd.DataFrame, table_name: str) -> pd.DataFrame:
    """
    Normaliza DataFrames ANTES de cargarlos a SQL.
    
    Estrategia: ESTANDARIZAR a 'Anio' (ASCII-safe, sin tildes)
    
    Aplica:
    - Conversión 'Año' → 'Anio' (elimina columna con tilde)
    - Normalización Gini (0-1)
    - Conversión Series → escalares
    - Normalización columnas deciles (D1-D10)
    """
    df = df.copy()
    
    # 1️⃣ Estandarizar a 'Anio' (ASCII-safe) - NO columnas duales
    if 'Año' in df.columns:
        if 'Anio' in df.columns:
            # Si ambas existen, eliminar 'Año' (con tilde)
            df = df.drop(columns=['Año'])
            print(f"   🔄 Eliminada columna 'Año' (ya existe 'Anio')")
        else:
            # Renombrar 'Año' → 'Anio'
            df = df.rename(columns={'Año': 'Anio'})
            print(f"   🔄 Renombrada 'Año' → 'Anio' (ASCII-safe)")
    
    # 2️⃣ Normalizar Gini a escala 0-1 si está en 0-100
    if 'Gini' in df.columns:
        try:
            max_gini = pd.to_numeric(df['Gini'], errors='coerce').max()
            if max_gini > 1:
                df['Gini'] = df['Gini'] / 100.0
                print(f"   🔄 Gini normalizado: 0-100 → 0-1 (max: {df['Gini'].max():.3f})")
        except Exception as e:
            print(f"   ⚠️ Error normalizando Gini: {e}")
    
    # 3️⃣ Conversión de Series a escalares en todas las columnas
    for col in df.columns:
        # Si la columna contiene objetos pandas (Series, Index, etc.)
        if df[col].dtype == 'object':
            df[col] = df[col].apply(lambda x: 
                x.item() if hasattr(x, 'item') and hasattr(x, '__len__') and len(x) == 1 
                else x.values[0] if hasattr(x, 'values') and hasattr(x, '__len__') and len(x) == 1
                else x
            )
    
    # 4️⃣ Normalizar nombres de columnas de deciles (D1_Renta → D1, Decil_1 → D1, etc.)
    from src.notebook_fixtures import normalize_decile_columns
    try:
        df = normalize_decile_columns(df)
    except Exception:
        pass  # Si no aplica, continuar
    
    return df

### 📤 Carga de Tablas a SQL Server

In [8]:
from datetime import datetime

print("="*80)
print("🚀 INICIANDO CARGA A SQL SERVER (CON NORMALIZACIÓN)")
print("="*80)
inicio = datetime.now()

# Contador de éxitos y errores
tablas_cargadas = 0
errores = []

for nombre_tabla, df in dataframes_a_cargar.items():
    try:
        print(f"\n📊 Cargando: {nombre_tabla}")
        print(f"   Registros ANTES: {len(df)}")
        
        # Evitar reemplazar tablas con DataFrames vacíos
        if df.empty:
            print(f"   ⚠️ Skipping {nombre_tabla} (DataFrame vacío)")
            continue

        # 🔧 NORMALIZACIÓN ANTES DE CARGAR A SQL
        df = normalize_for_sql(df, nombre_tabla)
        
        print(f"   Registros DESPUÉS: {len(df)}")
        print(f"   Columnas: {list(df.columns)[:5]}{'...' if len(df.columns) > 5 else ''}")

        # Escribir en SQL Server (reemplaza si existe)
        df.to_sql(
            name=nombre_tabla,
            con=engine,
            if_exists='replace',
            index=False,
            schema='dbo'
        )

        tablas_cargadas += 1
        print(f"   ✅ Cargada exitosamente")

    except Exception as e:
        errores.append((nombre_tabla, str(e)))
        print(f"   ❌ ERROR: {str(e)}")

# Resumen final
fin = datetime.now()
duracion = (fin - inicio).total_seconds()

print("\n" + "="*80)
print("📊 RESUMEN DE CARGA")
print("="*80)
print(f"✅ Tablas cargadas exitosamente: {tablas_cargadas}/{len(dataframes_a_cargar)}")

if errores:
    print(f"\n❌ Errores encontrados: {len(errores)}")
    for tabla, error in errores:
        print(f"   • {tabla}: {error}")
else:
    print("\n🎉 ¡Todas las tablas se cargaron correctamente!")

print(f"\n⏱️  Tiempo total: {duracion:.2f} segundos")
print(f"🕒 Finalizado: {fin.strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

🚀 INICIANDO CARGA A SQL SERVER (CON NORMALIZACIÓN)

📊 Cargando: INE_IPC_Nacional
   Registros ANTES: 24
   Registros DESPUÉS: 24
   Columnas: ['Anio', 'IPC_Medio_Anual', 'Inflacion_Anual_%']


   ✅ Cargada exitosamente

📊 Cargando: INE_Umbral_Pobreza_Hogar
   Registros ANTES: 32
   Registros DESPUÉS: 32
   Columnas: ['Anio', 'Tipo_Hogar', 'Umbral_Pobreza_Euros']
   ✅ Cargada exitosamente

📊 Cargando: INE_Carencia_Material_Decil
   Registros ANTES: 1683
   Registros DESPUÉS: 1683
   Columnas: ['Item', 'Anio', 'Valor', 'Decil']


   ✅ Cargada exitosamente

📊 Cargando: INE_AROPE_Edad_Sexo
   Registros ANTES: 408
   Registros DESPUÉS: 408
   Columnas: ['Anio', 'Sexo', 'Edad', 'Indicador', 'Valor']
   ✅ Cargada exitosamente

📊 Cargando: INE_AROPE_Hogar
   Registros ANTES: 308
   Registros DESPUÉS: 308
   Columnas: ['Anio', 'Tipo_Hogar', 'Indicador', 'Valor']
   ✅ Cargada exitosamente

📊 Cargando: INE_AROPE_Laboral
   Registros ANTES: 152
   Registros DESPUÉS: 152
   Columnas: ['Sexo', 'Situacion_Laboral', 'Territorio', 'Anio', 'AROPE']


   ✅ Cargada exitosamente

📊 Cargando: INE_AROPE_CCAA
   Registros ANTES: 0
   ⚠️ Skipping INE_AROPE_CCAA (DataFrame vacío)

📊 Cargando: INE_Gini_S80S20_CCAA
   Registros ANTES: 340
   Registros DESPUÉS: 340
   Columnas: ['Territorio', 'Anio', 'Gini', 'S80/S20']
   ✅ Cargada exitosamente

📊 Cargando: INE_Renta_Media_Decil
   Registros ANTES: 176
   Registros DESPUÉS: 176
   Columnas: ['Decil', 'Anio', 'Media', 'Mediana']
   ✅ Cargada exitosamente

📊 Cargando: INE_Poblacion_Edad_Sexo_Nacionalidad
   Registros ANTES: 0
   ⚠️ Skipping INE_Poblacion_Edad_Sexo_Nacionalidad (DataFrame vacío)

📊 Cargando: INE_Poblacion_Edad_Sexo_CCAA
   Registros ANTES: 0
   ⚠️ Skipping INE_Poblacion_Edad_Sexo_CCAA (DataFrame vacío)

📊 Cargando: INE_Gasto_Medio_Hogar_Quintil
   Registros ANTES: 5616
   Registros DESPUÉS: 5616
   Columnas: ['Anio', 'Quintil', 'Grupo_Gasto', 'Tipo_Valor', 'Valor']


   ✅ Cargada exitosamente

📊 Cargando: INE_IPC_Sectorial_ECOICOP
   Registros ANTES: 1248
   Registros DESPUÉS: 1248
   Columnas: ['Anio', 'Categoria_ECOICOP', 'Tipo_Metrica', 'IPC_Indice', 'Inflacion_Sectorial_%']


   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_Gini_Espana
   Registros ANTES: 10
   🔄 Gini normalizado: 0-100 → 0-1 (max: 0.346)
   Registros DESPUÉS: 10
   Columnas: ['Gini', 'age', 'age_label', 'geo_code', 'geo_name']...
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_Gini_UE27
   Registros ANTES: 10
   🔄 Gini normalizado: 0-100 → 0-1 (max: 0.308)
   Registros DESPUÉS: 10
   Columnas: ['Gini', 'age', 'age_label', 'geo_code', 'geo_name']...
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_Gini_Ranking
   Registros ANTES: 365
   🔄 Gini normalizado: 0-100 → 0-1 (max: 0.466)
   Registros DESPUÉS: 365
   Columnas: ['Gini', 'geo_code', 'geo_name', 'Anio']
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_AROP_Espana
   Registros ANTES: 10
   Registros DESPUÉS: 10
   Columnas: ['AROP_%', 'sex', 'sex_label', 'age', 'age_label']...
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_AROP_UE27
   Registros ANTES: 10
   Registros DESPUÉS: 10
   Columnas: ['AROP_%', 'sex', 'sex_label', 'age', 'age_la

   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_AROP_Ranking
   Registros ANTES: 397
   Registros DESPUÉS: 397
   Columnas: ['AROP_%', 'geo_code', 'geo_name', 'Anio']
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_S80S20_Espana
   Registros ANTES: 10
   Registros DESPUÉS: 10
   Columnas: ['S80S20_Ratio', 'age', 'age_label', 'sex', 'sex_label']...
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_S80S20_UE27
   Registros ANTES: 10
   Registros DESPUÉS: 10
   Columnas: ['S80S20_Ratio', 'age', 'age_label', 'sex', 'sex_label']...
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_S80S20_Ranking
   Registros ANTES: 400
   Registros DESPUÉS: 400
   Columnas: ['S80S20_Ratio', 'geo_code', 'geo_name', 'Anio']


   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_Brecha_Pobreza_Espana
   Registros ANTES: 10
   Registros DESPUÉS: 10
   Columnas: ['Brecha_Pobreza_%', 'sex', 'sex_label', 'age', 'age_label']...
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_Brecha_Pobreza_UE27
   Registros ANTES: 10
   Registros DESPUÉS: 10
   Columnas: ['Brecha_Pobreza_%', 'sex', 'sex_label', 'age', 'age_label']...
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_Brecha_Pobreza_Ranking
   Registros ANTES: 378
   Registros DESPUÉS: 378
   Columnas: ['Brecha_Pobreza_%', 'geo_code', 'geo_name', 'Anio']
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_Impacto_Redistributivo_Espana
   Registros ANTES: 21
   Registros DESPUÉS: 21
   Columnas: ['geo_code', 'geo_name', 'Anio', 'Gini_Antes_SinPensiones', 'Gini_Antes_ConPensiones']...
   ✅ Cargada exitosamente

📊 Cargando: EUROSTAT_Impacto_Redistributivo_UE27
   Registros ANTES: 15
   Registros DESPUÉS: 15
   Columnas: ['geo_code', 'geo_name', 'Anio', 'Gini_Antes_SinPensiones',